In [1]:
import time
import pandas as pd
import numpy as np
import re

In [2]:
docker_down_str = "/yap/src/yap/yap api"

In [3]:
def send_email(user='dsakaidf@gmail.com', pwd='d54k4idf!', recipient='shkasta@post.bgu.ac.il',
               subject='finish expirement', body='finish the expirement'):
    import smtplib

    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        # SMTP_SSL Example
        server_ssl = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server_ssl.ehlo()  # optional, called by login()
        server_ssl.login(gmail_user, gmail_pwd)
        # ssl server doesn't support or need tls, so don't call server_ssl.starttls()
        server_ssl.sendmail(FROM, TO, message)
        # server_ssl.quit()
        server_ssl.close()
        print('successfully sent the mail')
    except:
        print("failed to send mail")

In [4]:
def restart_docker():
    try:
        import os
        os.system('docker run --publish 8000:8000 --detach -it name:dockerfile')
        print('restart docker')
        try:
            send_email(body='finish restart docker')
        except:
            print('error in sending mail')
    except:
        print('cant restart docker')
        raise

In [5]:
from subprocess import PIPE, run
def out(command):
    result = run(command, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
    return result.stdout

In [6]:
def check_docker():
    output_status_docker = out('docker ps')
    if docker_down_str in output_status_docker:
        return True
    else:
        return False    

In [7]:
def yap_parser(text, number_tries=5):
        try:
            import requests
            with requests.Session() as s:
                localhost_yap = "http://localhost:8000/yap/heb/joint"
                data = '{{"text": "{}  "}}'.format(text).encode('utf-8')  # input string ends with two space characters
                headers = {'content-type': 'application/json'}
                response = s.get(url=localhost_yap, data=data, headers=headers)
                json_response = response.json()
            return json_response
        except:
            if number_tries > 0:
                if check_docker():
                    time.sleep(60)
                else:
                    time.sleep(5)
                    restart_docker()
                    time.sleep(60)
                return yap_parser(text,number_tries-1)
            else:
                print('the problematic text is')
                print(text)
                raise

In [8]:
file_type = 'thread'
field_name = 'title'

In [9]:
try:
    df = pd.read_csv(f'data/{file_type}.csv')
    df[f'{field_name}_dict'] = df[f'{field_name}'].apply(lambda x: yap_parser(re.sub('- FXP', '', x)))
    df.to_csv(f'{file_type}_with_yap.csv')
    send_email(body='finish yap expirement')
except Exception as e:
    print(e)
    send_email(body=f'Error yap exp: \n {e}')



successfully sent the mail
